# 1. Library 🤔

In [ ]:
import numpy as np
import pandas as pd
import time

from sklearn.cluster import KMeans
from numpy import dot
from numpy.linalg import norm
from tqdm import tqdm

from sklearn.metrics import ndcg_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import warnings
warnings.filterwarnings('ignore')

# 2. Custom Code

movielens 데이터를 쉽게 다룰 수 있도록  
Group Recommender System 관련 함수를 추가 정의했다.

MEAN, 0, NASH algorithm 을 사용한 Aggregation 함수 등이 존재한다.


In [ ]:
from oha_GRS import data_load_100k, clustering, Nash_Group_predict, Group_predict, miss_matching, get_NDCG, predict_trunSVD, matrix_factorization, Precision_recall




# 2. Code 🙆‍♂️

In [6]:
import pandas as pd

Total, R,T = data_load_100k('u1', True)
Total4 = pd.DataFrame(matrix_factorization(Total, steps=300)) # mF 매트릭스 계산

epoch: 0, time: 0.000000
epoch: 1, time: 3.963048
epoch: 2, time: 7.773999
epoch: 3, time: 11.354002
epoch: 4, time: 14.787051
epoch: 5, time: 18.692394
epoch: 6, time: 22.351055
epoch: 7, time: 25.727051
epoch: 8, time: 29.370921
epoch: 9, time: 33.354956
epoch: 10, time: 37.675927
epoch: 11, time: 41.834037
epoch: 12, time: 45.722984
epoch: 13, time: 49.869848
epoch: 14, time: 54.314862
epoch: 15, time: 58.473537
epoch: 16, time: 62.373489
epoch: 17, time: 66.043639
epoch: 18, time: 69.564593
epoch: 19, time: 73.024639
epoch: 20, time: 76.594591
epoch: 21, time: 80.364591
epoch: 22, time: 83.886590
epoch: 23, time: 87.562593
epoch: 24, time: 91.169593
epoch: 25, time: 94.956593
epoch: 26, time: 98.724645
epoch: 27, time: 102.354610
epoch: 28, time: 105.784641
epoch: 29, time: 109.220593
epoch: 30, time: 112.645639
epoch: 31, time: 116.095593
epoch: 32, time: 119.394632
epoch: 33, time: 122.841638
epoch: 34, time: 126.166632
epoch: 35, time: 129.515638
epoch: 36, time: 132.858590
epoc

In [ ]:
#Total1 = Total.fillna(0)                                      # 0으로 채우는 경우
#Total2 = Total.fillna(Total.mean())                           # mean 으로 채우는 경우
#Total3 = predict_trunSVD(Total)                               # SVD 정규화로 채우는 경우
#Total4 = pd.DataFrame(matrix_factorization(Total, steps=300)) # MF 매트릭스 계산

# 각각 6, 10, 14, 18로 클러스터링 했을 때
NDCG_RESULT = [[0,0,0],[0,0,0],[0,0,0],[0,0,0]] # k = 20, 50, 100 
P_RESULT = [[0,0,0],[0,0,0],[0,0,0],[0,0,0]]    # k =  5, 10,  30

cluster_num = [6,10,14,18]

for k in ['u1', 'u2', 'u3', 'u4', 'u5']:
    # 1. DATA LOAD
    Total, R,T = data_load_100k(k, True)

    for i in range(len(cluster_num)): #[6,10,14,18]
        
        # 2. CLUSTERING
        train_data, test_data = clustering(cluster_num[i], Total4, R, T)

        # 3. AGGREGATING

        #Cluster per group ( Avg, LM, NASH )
        #rating = Group_predict(train_data)                              # AVG
        #rating = Group_predict(train_data, nantoZero=True, tool='lm')   # LM
        rating = Nash_Group_predict(train_data)                          # NASH
        
        # 4. MISS-MATCHING
        pred_data, test_data =  miss_matching(rating,test_data)
        test_data = test_data.fillna(0)

        # 5. NDCG _ PRECISION 
        nk = [20, 50, 100]
        pk = [5, 10, 30]

        #print(f'Cluster가 {i}개 일 때 :::::')
        for j in range(len(nk)):
            ndcg = get_NDCG(pred_data, test_data, num=cluster_num[i], k=nk[j])
            pre = Precision_recall(pred_data, test_data, k=pk[j])
            #pre, rec = get_pre_rec(pred_data, test_data, k=3)
            #print(f'ndch@{nk[j]} : {ndcg:.4f}, pre@{pk[j]} : {pre:.4f}')

            NDCG_RESULT[i][j] += ndcg
            P_RESULT[i][j]  += pre

print(NDCG_RESULT)
print(P_RESULT)






In [11]:
def kfold(data):
    kluster = [6,10,14,18]
    k = [1,2,3]
    for i in range(4):
        for j in range(3):
            data[i][j] = data[i][j] / 5
            print(f'{kluster[i]} 클러스터링에서 해당 {k[j]} 번째 지표는 {data[i][j]:.4f}')
        print()

#kfold(NDCG_RESULT)
#print('and')
#kfold(P_RESULT)

6 클러스터링에서 해당 1 번째 지표는 0.0018
6 클러스터링에서 해당 2 번째 지표는 0.0026
6 클러스터링에서 해당 3 번째 지표는 0.0247

10 클러스터링에서 해당 1 번째 지표는 0.0019
10 클러스터링에서 해당 2 번째 지표는 0.0049
10 클러스터링에서 해당 3 번째 지표는 0.0268

14 클러스터링에서 해당 1 번째 지표는 0.0027
14 클러스터링에서 해당 2 번째 지표는 0.0052
14 클러스터링에서 해당 3 번째 지표는 0.0228

18 클러스터링에서 해당 1 번째 지표는 0.0041
18 클러스터링에서 해당 2 번째 지표는 0.0069
18 클러스터링에서 해당 3 번째 지표는 0.0231



---

MF 클러스터링 상태에서 AVG, LM, NASH 비교

### 1. AVG

Cluster가 6개 일 때 :::::  
ndch@20 : 0.0040, pre@5 : 0.0008  
ndch@50 : 0.0201, pre@10 : 0.0015  
ndch@100 : 0.0539, pre@30 : 0.0217  

Cluster가 10개 일 때 :::::  
ndch@20 : 0.0062, pre@5 : 0.0021  
ndch@50 : 0.0177, pre@10 : 0.0022  
ndch@100 : 0.0498, pre@30 : 0.0171  

Cluster가 14개 일 때 :::::  
ndch@20 : 0.0100, pre@5 : 0.0025  
ndch@50 : 0.0220, pre@10 : 0.0092  
ndch@100 : 0.0508, pre@30 : 0.0244

Cluster가 18개 일 때 :::::  
ndch@20 : 0.0104, pre@5 : 0.0037  
ndch@50 : 0.0221, pre@10 : 0.0059  



### 2. LM

Cluster가 6개 일 때 :::::  
ndch@20 : 0.0039, pre@5 : 0.0010  
ndch@50 : 0.0067, pre@10 : 0.0026  
ndch@100 : 0.0135, pre@30 : 0.0189

Cluster가 10개 일 때 :::::  
ndch@20 : 0.0062, pre@5 : 0.0013  
ndch@50 : 0.0105, pre@10 : 0.0059  
ndch@100 : 0.0202, pre@30 : 0.0226

Cluster가 14개 일 때 :::::  
ndch@20 : 0.0097, pre@5 : 0.0016  
ndch@50 : 0.0153, pre@10 : 0.0036  
ndch@100 : 0.0264, pre@30 : 0.0192

Cluster가 18개 일 때 :::::  
ndch@20 : 0.0106, pre@5 : 0.0011  
ndch@50 : 0.0169, pre@10 : 0.0049  
ndch@100 : 0.0302, pre@30 : 0.0203

### 3. NASH

Cluster가 6개 일 때 :::::  
ndch@20 : 0.0046, pre@5 : 0.0006  
ndch@50 : 0.0206, pre@10 : 0.0017  
ndch@100 : 0.0525, pre@30 : **0.0227**

Cluster가 10개 일 때 :::::  
ndch@20 : 0.0062, pre@5 : 0.0019  
ndch@50 : 0.0185, pre@10 : 0.0056  
ndch@100 : 0.0478, pre@30 : 0.0177

Cluster가 14개 일 때 :::::  
ndch@20 : 0.0098, pre@5 : 0.0049  
ndch@50 : 0.0226, pre@10 : 0.0075  
ndch@100 : **0.0533** , pre@30 : 0.0224

Cluster가 18개 일 때 :::::  
ndch@20 : 0.0103, pre@5 : 0.0042  
ndch@50 : 0.0222, pre@10 : 0.0052  
ndch@100 : 0.0517, pre@30 : 0.0246